In [ ]:
# current submission is .78
# Things I'd like to do:
    # Hyper Parameters for Linear Regression and Random Forest?
    # Can we use the cabin column in a meaningful way for rows that have that?
    # find other ways to handle NaN Age?
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, minmax_scale
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
#!kaggle competitions download -c titanic
#!kaggle datasets download -d zynicide/wine-reviews # Use this to get data if you want it.

In [ ]:
og_df = pd.read_csv('./data/train.csv', engine='python')
og_df

In [ ]:
# just rounds the models prediction to 0 or 1.
def get_model_val(df_row, model):
    if(model.predict([df_row])[0] > .5):
        return 1
    else:
        return 0

def clean_data(df):
    # Transformations here.
    new_df = df.copy() 
    new_df = new_df.drop(columns=['Name', 'Ticket', 'Cabin'])

    new_df['Age'] = new_df['Age'].fillna(new_df.Age.median())

    new_df = pd.get_dummies(new_df, columns=['Sex', 'Embarked', 'Pclass'])
    new_df = pd.DataFrame(minmax_scale(new_df), columns=new_df.columns)

    return new_df


In [ ]:
df = clean_data(og_df)
df_passenger_id = og_df['PassengerId'].copy()
df_labels = df['Survived'].copy()
corr_matrix = df.corr()
df = df.drop(columns=['Survived', 'PassengerId'])

In [ ]:
corr_matrix['Survived'].sort_values(ascending=False)

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(df,df_labels)
model = lin_reg
lin_reg.predict([df.iloc[1]])[0] # sanity check

In [ ]:
tree_reg = DecisionTreeRegressor()
tree_reg.fit(df, df_labels)
model = tree_reg
tree_reg.predict([df.iloc[1]])[0] # sanity check

In [ ]:
forest_reg = RandomForestRegressor()
forest_reg.fit(df, df_labels)
model = forest_reg
forest_reg.predict([df.iloc[1]])[0] # sanity check

In [ ]:
knn = KNeighborsClassifier(n_neighbors=4) # can play with this a bit.
knn.fit(df,df_labels)
model = knn
knn.predict([df.iloc[1]])[0] # sanity check

In [ ]:
tr_og_df = pd.read_csv('./data/test.csv', engine='python')
tr_df_passengerId = tr_og_df['PassengerId'].copy()
tr_df = clean_data(tr_og_df)
tr_df['Fare'] = tr_df['Fare'].fillna(tr_df.Fare.median())
tr_df = tr_df.drop(columns=['PassengerId'])
tr_df['Survived'] = tr_df.apply(lambda row: get_model_val(row, model), axis=1)

In [ ]:
final_df = pd.DataFrame(list(zip(tr_df_passengerId, tr_df['Survived'])),columns=['PassengerId', 'Survived'])

In [ ]:
final_df.to_csv('output.csv', index=False)
#!kaggle competitions submit -c titanic -f ./output.csv
#final_df = final_df.drop([417], axis="index") # We have to do this before testing for some reason.

In [ ]:
ans_df = pd.read_csv('./data/answers.csv', engine='python')
ans_df = ans_df.rename({1: 'PassengerId', 2:'Survived'}, axis='columns')
ans_df.drop(ans_df.index[0:892], inplace=True)

In [ ]:
# For local testing so I don't have to submit all the time.
correct = 0
total = 0 
for index, row in final_df.iterrows():
    if ans_df.loc()[row['PassengerId']]['survived'] == row['Survived']:
        correct = correct +1
    total += 1 
print("{} out of {} for a % of {}".format(correct, total, correct/total))